# 1: MTC MNL Mode Choice

In [1]:
import pandas as pd
import larix as lx

/Users/jeffnewman/LocalGit/larix/larix/model/numbamodel.py:24: UserWarning: 

##### larix is experimental, and not feature-complete  #####
the first time you import on a new system, this package will
compile optimized binaries for your machine, which may take 
a little while, please be patient 

  warnings.warn( ### EXPERIMENTAL ### )
OMP: Info #273: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [ ]:
# TEST
pd.set_option("display.max_columns", 999)
pd.set_option('expand_frame_repr', False)
pd.set_option('display.precision', 3)
from pytest import approx
from larix.util.testing import assert_same_text

This example is a mode choice model built using the MTC example dataset.
First we create the Dataset and Model objects:

In [ ]:
d = lx.examples.MTC(format='dataset')
d

In [ ]:
m = lx.Model(d)

Then we can build up the utility function.  We'll use some :ref:`idco` data first, using
the `Model.utility.co` attribute.  This attribute is a dict-like object, to which
we can assign :class:`LinearFunction` objects for each alternative code.

In [ ]:
from larix import P, X, PX
m.utility_co[2] = P("ASC_SR2")  + P("hhinc#2") * X("hhinc")
m.utility_co[3] = P("ASC_SR3P") + P("hhinc#3") * X("hhinc")
m.utility_co[4] = P("ASC_TRAN") + P("hhinc#4") * X("hhinc")
m.utility_co[5] = P("ASC_BIKE") + P("hhinc#5") * X("hhinc")
m.utility_co[6] = P("ASC_WALK") + P("hhinc#6") * X("hhinc")

Next we'll use some idca data, with the `utility_ca` attribute. This attribute
is only a single :class:`LinearFunction` that is applied across all alternatives
using :ref:`idca` data.  Because the data is structured to vary across alternatives,
the parameters (and thus the structure of the :class:`LinearFunction`) does not need
to vary across alternatives.

In [ ]:
m.utility_ca = PX("tottime") + PX("totcost")

Lastly, we need to identify :ref:`idca` data that gives the availability for each
alternative, as well as the number of times each alternative is chosen. (In traditional
discrete choice analysis, this is often 0 or 1, but it need not be binary, or even integral.)

In [ ]:
m.availability_var = 'avail'
m.choice_ca_var = 'chose'

And let's give our model a descriptive title.

In [ ]:
m.title = "MTC Example 1 (Simple MNL)"

We can view a summary of the choices and alternative 
availabilities to make sure the model is set up 
correctly.

In [ ]:
m.choice_avail_summary()

In [ ]:
# TEST
s = '''            name  chosen available
altid                                    
1                    DA    3637      4755
2                   SR2     517      5029
3                  SR3+     161      5029
4               Transit     498      4003
5                  Bike      50      1738
6                  Walk     166      1479
< Total All Alternatives > 5029          
'''
import re
mash = lambda x: re.sub('\s+', ' ', x).strip()
assert mash(s) == mash(str(m.choice_avail_summary()))

We'll set a parameter cap (bound) at +/- 20, which helps improve the 
numerical stability of the optimization algorithm used in estimation.

In [ ]:
m.set_cap(20)

Having created this model, we can then estimate it:

In [ ]:
# TEST
assert dict(m.required_data()) == {
    'ca': ['totcost', 'tottime'],
    'co': ['hhinc'],
    'choice_ca': 'chose',
    'avail_ca': 'avail',
}
assert m.loglike() == approx(-7309.600971749634)

In [ ]:
assert m.compute_engine == 'jax'

In [ ]:
result = m.maximize_loglike(stderr=True)

In [ ]:
# TEST
assert result.loglike == approx(-3626.18625551293)
assert result.logloss == approx(0.7210551313408093)
assert result.message == 'Optimization terminated successfully'

assert m.total_weight() == 5029.0

In [ ]:
m.parameter_summary()

In [ ]:
# TEST
summary = _
assert_same_text(
    summary.data.to_markdown(),
    '''
    | Parameter   |     Value |   Std Err |   t Stat | Signif   |   Null Value |
    |:------------|----------:|----------:|---------:|:---------|-------------:|
    | ASC_BIKE    | -2.38     |  0.304    |    -7.81 | ***      |            0 |
    | ASC_SR2     | -2.18     |  0.105    |   -20.82 | ***      |            0 |
    | ASC_SR3P    | -3.73     |  0.178    |   -20.97 | ***      |            0 |
    | ASC_TRAN    | -0.672    |  0.133    |    -5.07 | ***      |            0 |
    | ASC_WALK    | -0.207    |  0.194    |    -1.07 |          |            0 |
    | hhinc#2     | -0.00217  |  0.00155  |    -1.39 |          |            0 |
    | hhinc#3     |  0.000375 |  0.00254  |     0.15 |          |            0 |
    | hhinc#4     | -0.00527  |  0.00183  |    -2.88 | **       |            0 |
    | hhinc#5     | -0.0128   |  0.00532  |    -2.4  | *        |            0 |
    | hhinc#6     | -0.00968  |  0.00303  |    -3.19 | **       |            0 |
    | totcost     | -0.00492  |  0.000239 |   -20.59 | ***      |            0 |
    | tottime     | -0.0513   |  0.0031   |   -16.56 | ***      |            0 |
    '''
)

In [ ]:
print(summary.data.to_markdown())

It is a little tough to read this report because the parameters can show up 
in pretty much any order, as they are not sorted
when they are automatically discovered by Larch.
We can use the reorder method to fix this:

In [ ]:
m.ordering = (
    ("LOS", "totcost", "tottime", ),
    ("ASCs", "ASC.*", ),
    ("Income", "hhinc.*", ),
)

In [ ]:
m.parameter_summary()

In [ ]:
# TEST
summary2 = _
assert_same_text(
    summary2.data.to_markdown(),
    '''
    |                       |     Value |   Std Err |   t Stat | Signif   |   Null Value |
    |:----------------------|----------:|----------:|---------:|:---------|-------------:|
    | ('LOS', 'totcost')    | -0.00492  |  0.000239 |   -20.59 | ***      |            0 |
    | ('LOS', 'tottime')    | -0.0513   |  0.0031   |   -16.56 | ***      |            0 |
    | ('ASCs', 'ASC_BIKE')  | -2.38     |  0.304    |    -7.81 | ***      |            0 |
    | ('ASCs', 'ASC_SR2')   | -2.18     |  0.105    |   -20.82 | ***      |            0 |
    | ('ASCs', 'ASC_SR3P')  | -3.73     |  0.178    |   -20.97 | ***      |            0 |
    | ('ASCs', 'ASC_TRAN')  | -0.672    |  0.133    |    -5.07 | ***      |            0 |
    | ('ASCs', 'ASC_WALK')  | -0.207    |  0.194    |    -1.07 |          |            0 |
    | ('Income', 'hhinc#2') | -0.00217  |  0.00155  |    -1.39 |          |            0 |
    | ('Income', 'hhinc#3') |  0.000375 |  0.00254  |     0.15 |          |            0 |
    | ('Income', 'hhinc#4') | -0.00527  |  0.00183  |    -2.88 | **       |            0 |
    | ('Income', 'hhinc#5') | -0.0128   |  0.00532  |    -2.4  | *        |            0 |
    | ('Income', 'hhinc#6') | -0.00968  |  0.00303  |    -3.19 | **       |            0 |
    '''
)


In [ ]:
m.loglike_null()